# NCPT Paperspace One-click (v0.3)

### Last updated: 25 Dec 2023

Check for updates [here](https://github.com/NoCrypt/paperspace-ncpt)

**!! ONLY RUN IN JUPYTERLAB !!**

![](https://i.ibb.co/M9ZH6qf/image.png)




To hide the code, press on this blue bar on the left of the cell

![](https://i.ibb.co/gFqKVV6/chrome-23-12-25-195649.png)

---


#### 1. Configurations

In [ ]:
from ipywidgets import Checkbox, Text
from IPython.display import display, HTML
from pathlib import Path
from subprocess import check_output, check_call


style = {"description_width": "25%"}
layout = {"width": "70%"}

label_style = HTML(
    "<style> .widget-label, .widget-label-basic >  span { color: white !important; font-weight: bold;}  </style>"
)

persistent_config_control = Checkbox(
    value=False,  # Keep it false as default, presistent is good but I might change config in the future, also it's much more idiot-proof this way
    description="Persistent config",
    layout=layout,
    style=style,
)

persistence_control = Checkbox(
    value=False,  # Keep it false as default, I just prefer it off by default like in old colab
    description="Save outputs to persistant notebook",
    layout=layout,
    style=style,
)

force_cache_control = Checkbox(
    value=False,  # Keep it false as default, cuz why?!
    description="Force use of cached dependencies (faster, may break things)",
    layout=layout,
    style=style,
)

extn_update_control = Checkbox(
    value=True, # You would want newer extensions by default
    description="Update all extensions on launch",
    layout=layout,
    style=style,
)

# textboxes
hf_token = Text(
    value="",
    description="HuggingFace token (optional)",
    placeholder="hf_xyz",
    style=style,
    layout=layout,
)
commandline_arg_control = Text(
    value="--xformers",
    description="Extra commandline arguments",
    style=style,
    layout=layout,
)

# show them
display(
    persistent_config_control,
    persistence_control,
    force_cache_control,
    extn_update_control,
    hf_token,
    commandline_arg_control,
)

#### 2. Start 🚀

In [ ]:
import os
import zipfile
import time
import shutil
from subprocess import check_call
from pathlib import Path
from tempfile import NamedTemporaryFile
from urllib.parse import urlparse
from os import getenv, environ

# --------------------- Edit these if needed

model_urls = [
    "https://huggingface.co/NoCrypt/expermental_models/resolve/main/shux7.safetensors",
]

vae_urls = [
    "https://huggingface.co/NoCrypt/resources/resolve/main/VAE/any.vae.safetensors",
    "https://huggingface.co/NoCrypt/resources/resolve/main/VAE/blessed2.vae.safetensors",
    "https://huggingface.co/NoCrypt/resources/resolve/main/VAE/wd-blessed09.vae.safetensors",
    "https://huggingface.co/NoCrypt/resources/resolve/main/VAE/wd.vae.safetensors",
    "https://civitai.com/api/download/models/88156", # clear vae
]

lora_urls = [
    # "https://civitai.com/api/download/models/31228", # 3232 pixels
]

extensions_urls = [  
    #"https://github...",
]

# ---------------------

# options class for easy access
class CurrentOptions:
    # fixed (Always present) command line args, needed for this to work at all
    static_args = (
        "--enable-insecure-extension-access --disable-safe-unpickle --theme dark  --listen --port 6006"
    )

    config_file = ""

    @property
    def persistent_output(self) -> bool:
        return persistence_control.value
        
    @property
    def persistent_config(self) -> bool:
        return persistent_config_control.value
    
    @property
    def force_cache_deps(self) -> bool:
        return force_cache_control.value

    @property
    def update_all_extensions(self) -> bool:
        return extn_update_control.value

    @property
    def optional_huggingface_token(self) -> str:
        return hf_token.value
  
    @property
    def commandline_arguments(self) -> str:
        arg_list = [self.static_args]
        if hf_token.value != "":
            arg_list.extend(["--hf-token-out", hf_token.value])
        if extn_update_control.value is True:
            arg_list.append("--update-all-extensions")
        if persistent_config_control.value is True and self.config_file != "":
            arg_list.extend(["--ui-settings-file", self.config_file])
        if commandline_arg_control.value != "":
            arg_list.extend([x for x in commandline_arg_control.value.split(" ") if x != ""])
        return " ".join(arg_list)


# spawn the boi
options = CurrentOptions()

# paperspace-provided directories
notebook_dir = Path("/notebooks")
storage_dir = Path("/storage")

# local-only semi-persistent storage
content_dir = Path("/content")

# symlink the directory for easier access
content_dir.mkdir(exist_ok=True)
content_symlink = notebook_dir.joinpath("content")
if not all(
    (
        content_symlink.is_symlink(),
        content_symlink.resolve() == content_dir,
    )
):
    # recreate the symlink
    content_symlink.unlink(missing_ok=True)
    content_symlink.symlink_to(content_dir, target_is_directory=True)


# where we put deps
dep_cache_dir = storage_dir.joinpath("a1111_dependencies")

# where models will go
sdw_dir = content_dir.joinpath("sdw")
models_dir = sdw_dir.joinpath("models")
extensions_dir = sdw_dir.joinpath("extensions")
output_dir = notebook_dir.joinpath("outputs/") 

# options for aria2 to use
aria2_opts = [ "--console-log-level=error", "--summary-interval=60", "-j1", "-x1", "-s1", "-k1M", "-c"]

# download repo if needed
config_json = content_dir.joinpath("sdw/config.json")
persistent_config_file = notebook_dir.joinpath("config.json")

if not config_json.exists() :
    # repo isn't here so LET'S GOOOOOOOOOO
    aria2_task = "\n".join(
        [
            "https://huggingface.co/NoCrypt/fast-repo/resolve/main/repo.tar.lz4",
            "\tout=repo.tar.lz4",
            "https://huggingface.co/NoCrypt/fast-repo/resolve/main/cache.tar.lz4",
            "\tout=cache.tar.lz4",
        ]
    )

    with NamedTemporaryFile("w") as aria2_file:
        aria2_file.write(aria2_task)
        aria2_file.flush()
        display("Downloading repo and cache. This may take a while...")
        check_call(
            ["aria2c", f"--input-file={aria2_file.name}"] + aria2_opts,
            cwd=content_dir,
        )

    display("Extracting repo...")
    check_call(["tar", "-xI", "lz4", "-f", "repo.tar.lz4", "--directory=/"], cwd=content_dir)
    content_dir.joinpath("repo.tar.lz4").unlink()

    display("Extracting cache...")
    check_call(["tar", "-xI", "lz4", "-f", "cache.tar.lz4", "--directory=/"], cwd=content_dir)
    content_dir.joinpath("cache.tar.lz4").unlink()

if options.persistent_config :
    if not persistent_config_file.exists():
        !cp {config_json} {notebook_dir}
    config_json=persistent_config_file

options.config_file= str(config_json)

# deal with persistent output 
if options.persistent_output:
  
  if not output_dir.exists():
      output_dir.mkdir(exist_ok=True)

  !sed -i 's@"outdir_txt2img_samples": "outputs/txt2img-images"@"outdir_txt2img_samples": "{output_dir}/txt2img-images"@' {config_json}
  !sed -i 's@"outdir_img2img_samples": "outputs/img2img-images"@"outdir_img2img_samples": "{output_dir}/img2img-images"@' {config_json}
  !sed -i 's@"outdir_extras_samples": "outputs/extras-images"@"outdir_extras_samples": "{output_dir}/extras-images"@' {config_json}
  !sed -i 's@"outdir_txt2img_grids": "outputs/txt2img-grids"@"outdir_txt2img_grids": "{output_dir}/txt2img-grids"@' {config_json}
  !sed -i 's@"outdir_img2img_grids": "outputs/img2img-grids"@"outdir_img2img_grids": "{output_dir}/img2img-grids"@' {config_json}
  !sed -i 's@"outdir_save": "log/images"@"outdir_save": "{output_dir}/log/images"@' {config_json}

# set up dependency cache
if dep_cache_dir.exists() is False or options.force_cache_deps:
    if options.force_cache_deps and dep_cache_dir.exists():
        display("Forcing removal of dependency cache!")
        shutil.rmtree(dep_cache_dir)
    display("Caching dependencies. This may take a while...")
    dep_cache_dir.mkdir(exist_ok=True)
    check_call(
        ["bash", "./webui.sh", "-f", options.commandline_arguments, "--exit"],
        cwd=content_dir.joinpath("sdw"),
    )

    # recursively copy all files in /usr/local/lib/python3.10/site-packages/ to /storage/a1111_dependencies/ (except for .pyc files)
    if Path("/usr/local/lib/python3.10/site-packages/").exists():
        display("Copying dependencies to persistent cache...")
        shutil.copytree(
            "/usr/local/lib/python3.10/site-packages/",
            "/storage/a1111_dependencies/",
            symlinks=True,
            ignore=shutil.ignore_patterns("*.pyc"),
            dirs_exist_ok=True,
        )

# func to acquire checkpoints with
def acquire_checkpoints(checkpoint_urls: list[str], dst: Path = sdw_dir):
    aria2_task_lines = []
    for url in checkpoint_urls:
        if 'drive.google' in url:
            if 'folders' in url:
                !cd {dst} && gdown --folder {url} --fuzzy -c
            else:
                !cd {dst} && gdown {url} --fuzzy -c
        elif 'civitai' in url :
            aria2_task_lines.extend([url,""])
        else :
            filename = urlparse(url).path.split("/")[-1]
            if not dst.joinpath(filename).exists():
                aria2_task_lines.extend([url, f"\tout={filename}"])
    if len(aria2_task_lines) > 0:
        with NamedTemporaryFile("w") as aria2_file:
            aria2_file.write("\n".join(aria2_task_lines))
            aria2_file.flush()
            display(f"Downloading {len(aria2_task_lines)} checkpoints, this may take a while...")
            check_call(
                ["aria2c", f"--input-file={aria2_file.name}"] + aria2_opts,
                cwd=dst,
            )
        display("Done!")
    else:
        display("All checkpoints already downloaded!")

# func to acquire extensions 
def acquire_extensions(extensions_urls: list[str]):
    for url in extensions_urls:
        repo_name = url.split('/')[-1]
        if extensions_dir.joinpath(repo_name).exists():
            !cd {extensions_dir}/{repo_name} && git fetch && git merge
        else:
            !git clone {url} {extensions_dir}/{repo_name}
    display("All extensions acquired!")


# actually load them
acquire_checkpoints(model_urls,models_dir.joinpath("Stable-diffusion"))
acquire_checkpoints(vae_urls, models_dir.joinpath("VAE"))
acquire_checkpoints(lora_urls,models_dir.joinpath("Lora"))
acquire_extensions(extensions_urls)

# start. the thing
environ["COMMANDLINE_ARGS"] = options.commandline_arguments
environ["REQS_FILE"] = "requirements_versions.txt"

display("Starting 'tensorboard'...")
display(f"Access via this URL: https://tensorboard-{getenv('PAPERSPACE_FQDN')}")

webui_user = f"""#!/bin/bash
export COMMANDLINE_ARGS='{options.commandline_arguments}'
venv_dir='-'
"""
content_dir.joinpath("sdw/webui-user.sh").write_text(webui_user)
check_call(["/usr/bin/env", "bash", "./webui.sh", "-f"], cwd=content_dir.joinpath("sdw"))


In [ ]:
# code to update webui
!cd {sdw_dir} && git pull